**目标:** 本Notebook旨在通过交互式可视化的方式，帮助理解卫星通信系统中的核心概念，包括卫星覆盖、小区规划和频率复用。我们将使用Uber的H3地理空间索引库来建模和分析这些系统。

## 1. 环境准备与库导入

首先，我们需要导入本教学中所有必需的Python库。请确保你已经按照之前的说明，在你的虚拟环境中安装了 `jupyterlab`, `h3`, `folium`, `numpy`, `matplotlib`, `pandas`, `plotly`, 和 `ipywidgets`。

In [ ]:
# 导入核心库
import h3
import folium
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from ipywidgets import interact, widgets
from math import radians, cos, sin, asin, sqrt, acos, pi
import plotly.graph_objects as go
from ipywidgets import VBox
import networkx as nx
from ipywidgets import HBox, VBox, HTML
import math
import random


print("H3-py 库版本:", h3.__version__)

## 2. H3地理空间索引基础

在进入卫星通信的具体应用之前，我们先深入理解一下H3库。

### 1. H3库是用来做什么的？

H3是一个**地理空间索引系统**。简单来说，它将整个地球表面用一层层无缝拼接的**六边形网格**覆盖起来。它的核心目标是将复杂的地理坐标（经纬度）转换成简单、易于处理的**单元格ID（Cell ID）**，从而极大地简化和加速地理空间数据的分析。

### 2. H3库如何使用？

H3的基本工作流程通常分为三步：

1.  **编码 (Encode)**: 将真实的经纬度坐标点映射到一个特定大小（分辨率）的六边形单元格中，并获得该单元格的唯一ID。
2.  **分析 (Analyze)**: 直接对这些简单的单元格ID进行计算和分析，例如查找邻居、确定层级关系、聚合数据等。由于操作的是ID而不是复杂的几何图形，计算速度非常快。
3.  **解码 (Decode)**: 当需要可视化或获取地理信息时，再将单元格ID转换回地理坐标（如中心点或边界）。

### 3. H3库能够解决什么问题？

- **高效的空间聚合**: 快速地将海量的点数据（如车辆轨迹、用户打卡记录）聚合到六边形网格中，用于制作热力图或进行密度分析。
- **快速的邻近搜索**: 六边形有6个邻居，距离和方向都非常规整，使得查找附近区域的操作（如查找附近的所有车辆）变得极其高效。
- **统一的空间划分**: 为不规则的地理区域提供一个标准化的网格系统。在我们的课程中，我们将用它来模拟卫星波束覆盖下的小区，这比用圆形或方形更贴近实际。
- **多尺度分析**: H3的网格具有层级关系，可以轻松地在不同精度（分辨率）之间切换，实现从宏观到微观的分析。

接下来，我们将通过代码实际操作以上提到的几个核心功能。

In [ ]:
# H3基础功能演示
lat, lon = 39.9961554, 116.358103  # 北京邮电大学的坐标
resolution = 7 # H3分辨率 (0-15, 数字越大, 六边形越小)

# 1. 坐标转H3索引
center_h3_index = h3.latlng_to_cell(lat, lon, resolution)
print(f"北京邮电大学的H3索引 (分辨率 {resolution}): {center_h3_index}")

# 2. H3索引转坐标
center_coords = h3.cell_to_latlng(center_h3_index)
print(f"H3索引的中心坐标: {center_coords}")

# 3. 获取邻居
neighbors = h3.grid_disk(center_h3_index, 2) # 获取2层邻居
print(f"中心六边形的2层邻居数量: {len(neighbors)}")

# 4. 可视化H3网格
m = folium.Map(location=[lat, lon], zoom_start=10)

# 修正: h3.cell_to_boundary now returns (lat, lon) tuples.
# We need to manually create the GeoJSON structure with (lon, lat) order.
def create_geojson_polygon(boundary_coords):
    # Swap lat/lon for GeoJSON standard (lon, lat)
    return {
        'type': 'Polygon',
        'coordinates': [[(lon, lat) for lat, lon in boundary_coords]]
    }

# 绘制中心六边形 (蓝色)
center_boundary_coords = h3.cell_to_boundary(center_h3_index)
center_geojson = create_geojson_polygon(center_boundary_coords)
folium.GeoJson(center_geojson, style_function=lambda x: {'fillColor': 'blue', 'color': 'blue'}).add_to(m)

# 绘制邻居六边形 (红色)
for h3_index in neighbors:
    if h3_index != center_h3_index:
        boundary_coords = h3.cell_to_boundary(h3_index)
        neighbor_geojson = create_geojson_polygon(boundary_coords)
        folium.GeoJson(neighbor_geojson, style_function=lambda x: {'fillColor': 'red', 'color': 'red'}).add_to(m)

m

## 3. 交互式卫星覆盖区域建模 (Satellite Footprint)

卫星的**覆盖区域**（或称**足迹**）是指卫星信号能够有效到达的地球表面区域。它的大小主要由两个因素决定：

- **卫星高度 (Altitude)**: 越高的卫星覆盖范围越广。我们这里假设为地球同步轨道（GEO），高度约为35,786公里。
- **最小仰角 (Minimum Elevation Angle)**: 地面接收天线需要以一个最小的角度（仰角）对准卫星才能保证通信质量。仰角越小，覆盖范围越大，但边缘信号质量越差。

下面的代码将创建一个交互式地图，可以通过滑动条来调整**卫星的位置（经纬度）**和**地面站的最小仰角**，并实时观察卫星覆盖区域的变化。

In [ ]:
# 导入新增的库
import plotly.graph_objects as go
from ipywidgets import VBox

def get_satellite_footprint_hexagons(sat_lat, sat_lon, min_elevation_angle, resolution=4):
    """
    根据卫星位置和最小仰角，计算其在地球表面的H3覆盖六边形集合。
    现在同时返回计算出的半径 k。
    """
    SAT_ALTITUDE = 357.86
    EARTH_RADIUS = 6371
    gamma = np.arcsin((EARTH_RADIUS * np.cos(np.radians(min_elevation_angle))) / (EARTH_RADIUS + SAT_ALTITUDE))
    alpha = np.pi/2 - np.radians(min_elevation_angle) - gamma
    coverage_radius_km = EARTH_RADIUS * alpha
    center_h3 = h3.latlng_to_cell(sat_lat, sat_lon, resolution)
    avg_hex_radius = h3.average_hexagon_edge_length(resolution, unit='km')
    k = int(coverage_radius_km / avg_hex_radius / 1.5)
    footprint_hexagons = h3.grid_disk(center_h3, k)
    return footprint_hexagons, k

def plot_2d_map_view(sat_lon, sat_lat, elevation, hexagons):
    """
    绘制2D Folium地图视图
    """
    # --- 修正: 禁用地图的水平重复扩展 ---
    # 1. 创建一个不带默认图层的地图
    m = folium.Map(location=[sat_lat, sat_lon], zoom_start=2, world_copy_jump=True)
    # 2. 创建一个TileLayer，并设置 no_wrap=True
    folium.TileLayer(
        tiles='CartoDB dark_matter',
        no_wrap=True  # 关键参数：禁用地图的水平重复
    ).add_to(m)
    # --- 修正结束 ---

    folium.Marker(
        [sat_lat, sat_lon], 
        popup=f'Satellite Position: ({sat_lat:.2f}, {sat_lon:.2f})', 
        icon=folium.Icon(color='red', icon='satellite', prefix='fa')
    ).add_to(m)
    geo_json_features = [{'type': 'Feature','geometry': {'type': 'Polygon','coordinates': [[(lon, lat) for lat, lon in h3.cell_to_boundary(h)]]},} for h in hexagons]
    geo_json_data = {'type': 'FeatureCollection', 'features': geo_json_features}
    folium.GeoJson(geo_json_data, style_function=lambda x: {'fillColor': 'blue', 'color': 'blue', 'weight': 1, 'fillOpacity': 0.3}).add_to(m)
    return m

# --- 带有3D小区绘制的视图函数 ---
def plot_3d_globe_view(sat_lon, sat_lat, elevation, hexagons, k):
    """
    使用 Plotly 绘制交互式3D地球视图，并在地球表面绘制H3小区。
    """
    # 1. 绘制地球球体
    theta = np.linspace(0, 2*np.pi, 100)
    phi = np.linspace(0, np.pi, 100)
    x_earth = 6371 * np.outer(np.cos(theta), np.sin(phi))
    y_earth = 6371 * np.outer(np.sin(theta), np.sin(phi))
    z_earth = 6371 * np.outer(np.ones(100), np.cos(phi))
    earth = go.Surface(x=x_earth, y=y_earth, z=z_earth, colorscale=[[0, 'lightblue'], [1, 'blue']], showscale=False, name="Earth")

    # 2. 计算卫星的3D笛卡尔坐标
    SAT_ALTITUDE = 35786
    r_sat = 6371 + SAT_ALTITUDE
    lat_rad, lon_rad = np.radians(sat_lat), np.radians(sat_lon)
    x_sat, y_sat, z_sat = r_sat*np.cos(lat_rad)*np.cos(lon_rad), r_sat*np.cos(lat_rad)*np.sin(lon_rad), r_sat*np.sin(lat_rad)
    satellite = go.Scatter3d(x=[x_sat], y=[y_sat], z=[z_sat], mode='markers', marker=dict(size=5, color='red'), name='Satellite')

    # 3. 绘制表示波束的圆锥
    beam_lines_x, beam_lines_y, beam_lines_z = [], [], []
    if k > 0:
        center_h3 = h3.latlng_to_cell(sat_lat, sat_lon, 3)
        outer_ring = h3.grid_ring(center_h3, k)
    else:
        outer_ring = hexagons

    for h in outer_ring:
        if h in hexagons:
            lat, lon = h3.cell_to_latlng(h)
            lat_r, lon_r = np.radians(lat), np.radians(lon)
            r_earth = 6371
            x_p, y_p, z_p = r_earth*np.cos(lat_r)*np.cos(lon_r), r_earth*np.cos(lat_r)*np.sin(lon_r), r_earth*np.sin(lat_r)
            beam_lines_x.extend([x_sat, x_p, None])
            beam_lines_y.extend([y_sat, y_p, None])
            beam_lines_z.extend([z_sat, z_p, None])
    
    beam = go.Scatter3d(x=beam_lines_x, y=beam_lines_y, z=beam_lines_z, mode='lines', line=dict(color='yellow', width=2), name='Beam')

    hex_lines_x, hex_lines_y, hex_lines_z = [], [], []
    r_hex = 6371.1 
    for h in hexagons:
        boundary = h3.cell_to_boundary(h)
        path = list(boundary) + [boundary[0]]
        for lat, lon in path:
            lat_r, lon_r = np.radians(lat), np.radians(lon)
            x_p = r_hex * np.cos(lat_r) * np.cos(lon_r)
            y_p = r_hex * np.cos(lat_r) * np.sin(lon_r)
            z_p = r_hex * np.sin(lat_r)
            hex_lines_x.append(x_p)
            hex_lines_y.append(y_p)
            hex_lines_z.append(z_p)
        hex_lines_x.append(None)
        hex_lines_y.append(None)
        hex_lines_z.append(None)
        
    hex_trace = go.Scatter3d(x=hex_lines_x, y=hex_lines_y, z=hex_lines_z, mode='lines', line=dict(color='blue', width=1), name='Footprint Cells')

    # 4. 创建图形并将所有元素添加到数据中
    fig = go.Figure(data=[earth, satellite, beam, hex_trace])
    fig.update_layout(
        title_text='3D Satellite View',
        scene=dict(
            xaxis=dict(title='X (km)', showticklabels=False),
            yaxis=dict(title='Y (km)', showticklabels=False),
            zaxis=dict(title='Z (km)', showticklabels=False),
            aspectmode='data'
        ),
        margin=dict(l=0, r=0, b=0, t=40)
    )
    return fig

# --- 主交互函数 ---
def display_all_views(sat_lon, sat_lat, elevation):
    hexagons, k = get_satellite_footprint_hexagons(sat_lat, sat_lon, elevation, resolution=3)
    map_2d = plot_2d_map_view(sat_lon, sat_lat, elevation, hexagons)
    globe_3d = plot_3d_globe_view(sat_lon, sat_lat, elevation, hexagons, k)
    
    print("--- 2D Map View ---")
    display(map_2d)
    
    print("\n--- 3D Globe View ---")
    globe_3d.show()

# 创建交互式控件
interact(
    display_all_views, 
    sat_lon=widgets.FloatSlider(min=-180, max=180, step=5, value=116.4, description='卫星经度'),
    sat_lat=widgets.FloatSlider(min=-90, max=90, step=5, value=39.9, description='卫星纬度'),
    elevation=widgets.IntSlider(min=5, max=45, step=1, value=10, description='最小仰角')
);

## 4. 蜂窝小区规划与频率复用

卫星通信系统通常采用**多波束（Multi-beam）**技术，将覆盖区域划分为多个更小的**小区（Cell）**，就像地面移动通信的蜂窝网络一样。这样做的好处是：

- **提高系统容量**: 可以在不同的小区中**复用**相同的频率，从而服务更多的用户。
- **提高信号质量**: 集中功率为更小的区域提供服务。

H3的六边形网格是模拟蜂窝网络的理想工具。我们将演示如何在一个给定的区域创建蜂窝网络，并应用**频率复用模式**。

**频率复用因子 (Frequency Reuse Factor, N)**: 指的是一组可用的总频率被分成多少个子集，在相邻的小区中使用不同的频率子集以避免干扰。常见的复用因子有3, 4, 7。

In [ ]:
def plot_cellular_network(center_lat, center_lon, grid_radius, resolution, reuse_factor):
    """
    可视化蜂窝网络和频率复用模式，并高亮显示中心簇及其频率编号。
    该版本完全不使用H3库，手动构建六边形网格以确保与通信理论一致。
    """
    
    # --- 1. 手动构建六边形网格 ---
    
    # 根据分辨率定义六边形的大小（单位：度）
    # 这个值可以调整以匹配H3在特定分辨率下的视觉大小
    hex_size = 3 / (2**(resolution - 6))

    def axial_to_latlon(q, r, center_lat, center_lon, size):
        """将轴坐标(q,r)转换为地理坐标"""
        # 简单的线性转换，在小范围内足够精确
        x = size * (3./2 * q)
        y = size * (np.sqrt(3)/2 * q + np.sqrt(3) * r)
        # 将笛卡尔偏移量转换为经纬度偏移量
        lon = center_lon + (x / (111.32 * np.cos(np.radians(center_lat))))
        lat = center_lat + (y / 111.32)
        return lat, lon

    def get_hexagon_corners(lat, lon, size):
        """获取一个六边形的所有顶点"""
        corners = []
        for i in range(6):
            angle_deg = 60 * i
            angle_rad = np.pi / 180 * angle_deg
            x = size * np.cos(angle_rad)
            y = size * np.sin(angle_rad)
            corner_lon = lon + (x / (111.32 * np.cos(np.radians(lat))))
            corner_lat = lat + (y / 111.32)
            corners.append((corner_lon, corner_lat))
        return corners

    # 生成网格内的所有轴坐标 (q,r)
    axial_coords = []
    for q in range(-grid_radius, grid_radius + 1):
        for r in range(-grid_radius, grid_radius + 1):
            if abs(q + r) <= grid_radius:
                axial_coords.append((q, r))

    # --- 2. 为每种N值定义其专属的、正确的着色公式 ---
    CLUSTER_DEFINITIONS = {
        1: {'formula': lambda q, r: 0},
        3: {'formula': lambda q, r: (q - r) % 3},
        4: {'formula': lambda q, r: (q % 2) * 2 + (r % 2)},
        7: {'formula': lambda q, r: (q - 2*r) % 7}
    }
    coloring_formula = CLUSTER_DEFINITIONS.get(reuse_factor, CLUSTER_DEFINITIONS[7])['formula']

    # --- 3. 为所有小区分配颜色 ---
    coord_colors = {}
    coord_color_indices = {}
    for q, r in axial_coords:
        color_index = coloring_formula(q, r)
        rgb = plt.get_cmap('viridis', reuse_factor)(color_index)[:3]
        hex_color = '#%02x%02x%02x' % (int(rgb[0]*255), int(rgb[1]*255), int(rgb[2]*255))
        coord_colors[(q, r)] = hex_color
        coord_color_indices[(q, r)] = color_index

    m = folium.Map(location=[center_lat, center_lon], zoom_start=10, tiles='CartoDB positron')
    
    # --- 4. 绘制所有着色的小区 ---
    for (q, r), color in coord_colors.items():
        lat, lon = axial_to_latlon(q, r, center_lat, center_lon, hex_size)
        corners = get_hexagon_corners(lat, lon, hex_size)
        folium.Polygon(locations=[(c[1], c[0]) for c in corners], color='black', weight=1, fillColor=color, fill_opacity=0.7).add_to(m)

    # --- 5. 识别、高亮中心簇并添加数字 ---
    if reuse_factor > 1:
        # 定义基础簇的形状
        base_cluster_shapes = {
            3: {(0,0), (1,0), (0,1)},
            4: {(0,0), (1,0), (0,1), (1,-1)}, # 修正为正确的菱形
            7: {(0,0), (1,0), (0,1), (-1,1), (-1,0), (0,-1), (1,-1)}
        }
        base_cluster_shape = base_cluster_shapes.get(reuse_factor, set())

        for q, r in base_cluster_shape:
            lat, lon = axial_to_latlon(q, r, center_lat, center_lon, hex_size)
            corners = get_hexagon_corners(lat, lon, hex_size)
            folium.Polygon(locations=[(c[1], c[0]) for c in corners], color='red', weight=4, fill=False).add_to(m)
            
            number = coord_color_indices.get((q, r), 0) + 1
            icon_html = f'<div style="font-family: Arial, sans-serif; font-size: 10pt; font-weight: bold; color: white; text-shadow: -1px -1px 0 #000, 1px -1px 0 #000, -1px 1px 0 #000, 1px 1px 0 #000;">{number}</div>'
            folium.Marker(location=[lat, lon], icon=folium.DivIcon(icon_size=(0,0), icon_anchor=(5,12), html=icon_html)).add_to(m)

    display(m)

# --- 6. 更新交互式控件 ---
interact(
    plot_cellular_network, 
    center_lat=widgets.FloatText(value=31.2, description='中心纬度'),
    center_lon=widgets.FloatText(value=121.4, description='中心经度'),
    grid_radius=widgets.IntSlider(min=1, max=10, step=1, value=5, description='网络半径'),
    resolution=widgets.IntSlider(min=5, max=8, step=1, value=6, description='小区分辨率'),
    reuse_factor=widgets.SelectionSlider(options=[1, 3, 4, 7], value=3, description='频率复用因子')
);

In [ ]:
def compare_reuse_patterns_n3(center_lat, center_lon, grid_radius, resolution):
    """
    对比N=3时，“不规则”与“规则”频率复用的区别。
    并排显示两个地图，并分析“最近同频小区”之间的距离。
    """
    reuse_factor = 3 # 固定N=3
    
    # --- 1. 手动构建六边形网格 ---
    hex_size = 3 / (2**(resolution - 6))

    def axial_to_latlon(q, r, center_lat, center_lon, size):
        x = size * (3./2 * q); y = size * (np.sqrt(3)/2 * q + np.sqrt(3) * r)
        lon = center_lon + (x / (111.32 * np.cos(np.radians(center_lat))))
        lat = center_lat + (y / 111.32)
        return lat, lon

    def get_hexagon_corners(lat, lon, size):
        corners = []
        for i in range(6):
            angle_deg = 60 * i; angle_rad = np.pi / 180 * angle_deg
            x = size * np.cos(angle_rad); y = size * np.sin(angle_rad)
            corner_lon = lon + (x / (111.32 * np.cos(np.radians(lat))))
            corner_lat = lat + (y / 111.32)
            corners.append((corner_lon, corner_lat))
        return corners

    axial_coords = []
    for q in range(-grid_radius, grid_radius + 1):
        for r in range(-grid_radius, grid_radius + 1):
            if abs(q + r) <= grid_radius:
                axial_coords.append((q, r))

    # --- 2. 计算两种模式下的颜色分配 ---
    colors = plt.get_cmap('viridis', reuse_factor)
    
    # a) 不规则模式 (手动实现的、正确的随机贪心着色)
    adj = {coord: [] for coord in axial_coords}
    coord_set = set(axial_coords)
    for q, r in axial_coords:
        for dq, dr in [(1,0), (0,1), (-1,1), (-1,0), (0,-1), (1,-1)]:
            if (q + dq, r + dr) in coord_set:
                adj[(q,r)].append((q + dq, r + dr))
    
    nodes_to_color = list(axial_coords)
    random.shuffle(nodes_to_color) # 关键：随机化节点顺序
    
    coloring_irregular = {}
    for node in nodes_to_color:
        neighbor_colors = {coloring_irregular[neighbor] for neighbor in adj[node] if neighbor in coloring_irregular}
        for color in range(reuse_factor):
            if color not in neighbor_colors:
                coloring_irregular[node] = color
                break
    
    # b) 规则模式 (N=3的数学公式)
    coloring_regular = {(q,r): (q - r) % 3 for q, r in axial_coords}

    # --- 3. 创建并排的两个地图 ---
    m_irregular = folium.Map(location=[center_lat, center_lon], zoom_start=10, tiles='CartoDB positron')
    m_regular = folium.Map(location=[center_lat, center_lon], zoom_start=10, tiles='CartoDB positron')

    # --- 4. 绘制小区并分析同频距离 ---
    analysis_results = ""
    target_color_index = 0 

    for coloring, map_obj, title in [(coloring_irregular, m_irregular, "不规则复用"), (coloring_regular, m_regular, "规则复用")]:
        for (q, r), color_index in coloring.items():
            lat, lon = axial_to_latlon(q, r, center_lat, center_lon, hex_size)
            corners = get_hexagon_corners(lat, lon, hex_size)
            rgb = colors(color_index)[:3]
            hex_color = '#%02x%02x%02x' % (int(rgb[0]*255), int(rgb[1]*255), int(rgb[2]*255))
            folium.Polygon(locations=[(c[1], c[0]) for c in corners], color='black', weight=1, fillColor=hex_color, fill_opacity=0.7).add_to(map_obj)
        
        target_cells = [coord for coord, c_idx in coloring.items() if c_idx == target_color_index]
        distances = []
        drawn_pairs = set()

        for i in range(len(target_cells)):
            q1, r1 = target_cells[i]
            min_dist = float('inf')
            for j in range(len(target_cells)):
                if i == j: continue
                q2, r2 = target_cells[j]
                dist = math.sqrt((q1-q2)**2 + (r1-r2)**2 + (q1-q2)*(r1-r2))
                if dist < min_dist: min_dist = dist
            
            for j in range(len(target_cells)):
                if i == j: continue
                q2, r2 = target_cells[j]
                dist = math.sqrt((q1-q2)**2 + (r1-r2)**2 + (q1-q2)*(r1-r2))
                if abs(dist - min_dist) < 1e-9:
                    pair = tuple(sorted(((q1, r1), (q2, r2))))
                    if pair not in drawn_pairs:
                        p1_latlon = axial_to_latlon(q1, r1, center_lat, center_lon, hex_size)
                        p2_latlon = axial_to_latlon(q2, r2, center_lat, center_lon, hex_size)
                        folium.PolyLine(locations=[p1_latlon, p2_latlon], color='red', weight=2.5, opacity=1).add_to(map_obj)
                        distances.append(dist)
                        drawn_pairs.add(pair)
        
        avg_dist = np.mean(distances) if distances else 0
        std_dist = np.std(distances) if distances else 0
        analysis_results += f"<b>{title}:</b><br>找到 {len(drawn_pairs)} 个最近同频对。<br>平均距离: {avg_dist:.2f} (单位:格)<br>距离标准差: {std_dist:.2f}<br><hr>"

    # --- 5. 显示结果 ---
    html_irregular = m_irregular._repr_html_()
    html_regular = m_regular._repr_html_()
    map_layout = {'width': '500px', 'height': '500px'}
    
    display(VBox([
        HTML(f"<h3>对比 N=3 频率复用模式 (仅显示颜色1的连接)</h3><p>{analysis_results}</p>"),
        HBox([
            VBox([HTML("<h4>不规则复用 (使用随机贪心)</h4>"), HTML(html_irregular, layout=map_layout)]),
            VBox([HTML("<h4>规则复用</h4>"), HTML(html_regular, layout=map_layout)])
        ])
    ]))

# --- 6. 创建交互式控件 ---
interact(
    compare_reuse_patterns_n3, 
    center_lat=widgets.FloatText(value=31.2, description='中心纬度'),
    center_lon=widgets.FloatText(value=121.4, description='中心经度'),
    grid_radius=widgets.IntSlider(min=1, max=10, step=1, value=5, description='网络半径'),
    resolution=widgets.IntSlider(min=5, max=8, step=1, value=6, description='小区分辨率')
);

## 5. 总结

通过本Notebook的交互式演示，我们学习了：

1.  **H3基础**: 如何使用H3来表示和操作地理空间数据。
2.  **卫星覆盖分析**: 如何根据卫星参数（位置、仰角）来建模其服务区域。
3.  **蜂窝网络规划**: 如何利用H3的六边形网格来设计蜂窝小区并应用频率复用方案。

**下一步可以探索的方向**: 
- **动态轨道模拟**: 模拟低地球轨道（LEO）卫星（如Starlink）的快速移动及其覆盖区域的动态变化。
- **链路预算分析**: 在每个H3六边形上计算信号强度、信噪比（SNR）等关键链路指标。
- **干扰分析**: 建模不同小区之间的同频干扰，并优化频率分配方案。